In [ ]:
import json
import pandas as pd
import requests

import qrisklab

pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_rows', None)  # Set max_rows to None to display all rows
pd.set_option('display.max_columns', None)  # Set max_columns to None to display all columns


In [ ]:
# Load stress parameters
file_path = './parameters/'

with open(file_path + 'spot_vol_shocks.json', 'r') as f:
    spot_vol_shocks_param = json.load(f)

crypto_parameters = qrisklab.core.utils.CryptoParameters(spot_vol_shocks_param)

with open(file_path + 'vol_surface_shocks.json', 'r') as f:
    vol_surface_shocks_param = json.load(f)

vol_surface_shocks_parameters = qrisklab.core.utils.VolSurfaceParameters(vol_surface_shocks_param)


In [12]:
# Dummy positions
df = pd.DataFrame([{'instrument': 'BTC-27DEC24', 'quantity': 100},
 {'instrument': 'BTC-27DEC24', 'quantity': 100},
 {'instrument': 'BTC-PERPETUAL', 'quantity': 500},
 {'instrument': 'BTC-PERPETUAL', 'quantity': 500},
 {'instrument': 'BTC-25OCT24-44000-P', 'quantity': -100},
 {'instrument': 'BTC-25OCT24-45000-P', 'quantity': -100},
 {'instrument': 'BTC-27DEC24-100000-C', 'quantity': -100},
 {'instrument': 'BTC-27DEC24-105000-C', 'quantity': -100},
 {'instrument': 'BTC_USDT', 'quantity': 100}])

In [ ]:
# load position data

df = qrisklab.crypto.fetch_market_data(df,'2024-08-13')

1580 instruments have been crawled.


In [14]:
df=df[['instrument','underlying','expiry','strike','put_call','multiplier',
    'quantity','time_to_expiry','rate','cost_of_carry_rate','vol','spot','atm_ivol']]

## Spot Vol Shocks

In [15]:
spot_vol_shocks = portstress.crypto.CryptoSpotVolShocks(crypto_parameters)

In [16]:
post_stress, col = spot_vol_shocks.apply_spot_vol_shocks(df)

In [ ]:
qrisklab.core.black_scholes.bs_pricing(strike=50000, time_to_expiry=0.232877, spot=54681.59, rate=0.03, vol=0.5775, put_call='put')
qrisklab.core.black_scholes.calc_vega(strike=50000, time_to_expiry=0.232877, spot=54681.59, rate=0.03, vol=0.5775, put_call='put')

# np.float64(3606.873391368419)
# np.float64(3700.518323796012)

np.float64(93.56524889234046)

In [18]:
for k in crypto_parameters.crypto_shocks.keys():
    print(f"{k}: {int(post_stress[k].sum()):,}")


Mt. Gox Hack - Feb 2014: -52,165,085
DAO Hack - Jun 2016: -30,086,822
2017-2018 Bear Market - Jan 2018: -94,299,953
COVID-19 Market Crash - Mar 2020: -72,480,840
China Crypto Ban - May 2021: -42,732,703
Elon Musk Tweets (Tesla U-Turn) - May 2021: -42,663,661
2021 Summer Correction - Jun 2021: -57,417,866
China Mining Crackdown - Jun 2021: -72,180,757
Terra (LUNA) Collapse - May 2022: -57,360,742
FTX Bankruptcy - Nov 2022: -36,112,709
Fear Avalanche: -36,930,612
Fear Avalanche Spot: -35,156,614
Bullish Frenzy: 33,400,103
Bullish Calm: 13,940,855
Bearish Fade: -13,764,925
BTC Leads: 13,553,022
ETH Leads: 1,194,598
BTC Spot 1%: 1,382,212
BTC Vol 1%: -27,013
spot -16% Vol -25%: -22,033,780
spot -16% Vol 0%: -22,348,894
spot -16% Vol +50%: -23,260,970
spot 16% Vol -25%: 22,394,220
spot 16% Vol 0%: 21,954,231
spot 16% Vol +50%: 20,690,928
spot -66% Vol +50%: -95,775,577
spot -33% Vol +50%: -47,326,592
spot +50% Vol +50%: 66,157,040
spot +100% Vol +50%: 131,727,050
spot +200% Vol +50%: 260,56

## Vol Surface Shocks

In [ ]:
vol_surface_shocks = qrisklab.crypto.CryptoVolSurfaceShocks(vol_surface_shocks_parameters)

In [20]:
import datetime
valuation_date = '2024-08-19'

df_options = df[df.put_call.isin(['put','call'])].copy()

re = vol_surface_shocks.run(
    df_options,
    valuation_date=datetime.datetime.strptime(valuation_date, '%Y-%m-%d').date())
# be careful about valuation date!!! By default it will use today()

pd.DataFrame(re)

,product,measure,value,group,liquidity,type
0,BTC,Parallel,"-19,721.6508",None,False,ix
1,BTC,TermStructure,-126.4170,None,False,ix
2,BTC,Skew,"-3,502.8397",None,False,ix
3,BTC,BidAsk,"-12,960.7810",None,False,ix
4,BTC,Sum,"-36,311.6886",None,False,ix
5,Sum,Parallel,"-19,721.6508",None,False,ix
6,Sum,TermStructure,-126.4170,None,False,ix
7,Sum,Skew,"-3,502.8397",None,False,ix
8,Sum,Sum,"-23,350.9076",None,False,ix
